In [20]:
! pip install tweepy
! pip install numpy
! pip install pandas
! pip install torch

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [56]:
! pip install detoxify

  Using cached detoxify-0.2.2-py3-none-any.whl (11 kB)
     |████████████████████████████████| 1.2 MB 410 kB/s eta 0:00:01
     |████████████████████████████████| 804.1 MB 2.4 kB/s eta 0:00:01    |█                               | 22.7 MB 1.7 MB/s eta 0:07:38     |██▎                             | 56.1 MB 2.1 MB/s eta 0:05:57     |███▏                            | 78.4 MB 2.1 MB/s eta 0:05:51     |████████▊                       | 220.1 MB 2.5 MB/s eta 0:03:53     |██████████                      | 249.1 MB 2.5 MB/s eta 0:03:39     |███████████▉                    | 297.1 MB 2.4 MB/s eta 0:03:31     |█████████████                   | 326.2 MB 2.0 MB/s eta 0:03:58     |█████████████▊                  | 343.8 MB 2.6 MB/s eta 0:02:59     |███████████████▍                | 386.7 MB 2.5 MB/s eta 0:02:45     |████████████████▍               | 411.5 MB 1.9 MB/s eta 0:03:23     |███████████████████▎            | 485.7 MB 2.3 MB/s eta 0:02:19     |█████████████████████▏          | 531.5 MB 1.8 

In [27]:
import os
import re
import tweepy
import numpy as np
import pandas as pd

In [7]:
from detoxify import Detoxify

In [8]:
TWITTER_API_KEY = os.environ.get('TWITTER_API_KEY')
TWITTER_API_SECRET = os.environ.get('TWITTER_API_SECRET')

In [9]:
auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET)

In [10]:
api = tweepy.API(auth)

In [11]:
tp = api.get_user('franciscojarceo')

In [12]:
tp._json['name']

'Francisco Javier Arceo'

In [13]:
def get_all_tweets(screen_name):
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200, max_id=oldest)
        #save most recent tweets
        alltweets.extend(new_tweets)
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print(f"...{len(alltweets)} tweets downloaded so far")

    return alltweets

In [14]:
mytweets = get_all_tweets('franciscojarceo')

getting tweets before 1370572974085058559
...396 tweets downloaded so far
getting tweets before 1366097134504796163
...596 tweets downloaded so far
getting tweets before 1362465423136972800
...795 tweets downloaded so far
getting tweets before 1359208814202019841
...994 tweets downloaded so far
getting tweets before 1356630343391002623
...1193 tweets downloaded so far
getting tweets before 1354471901167554566
...1393 tweets downloaded so far
getting tweets before 1352590445164027906
...1592 tweets downloaded so far
getting tweets before 1351252276078374911
...1791 tweets downloaded so far
getting tweets before 1349203559301193727
...1990 tweets downloaded so far
getting tweets before 1347691286174822402
...2189 tweets downloaded so far
getting tweets before 1345232393666490368
...2389 tweets downloaded so far
getting tweets before 1342152007289774080
...2589 tweets downloaded so far
getting tweets before 1340520723949842436
...2789 tweets downloaded so far
getting tweets before 1338632

In [15]:
mytweets[2]._json['in_reply_to_status_id']

In [16]:
mytweets[1]._json['in_reply_to_status_id']

In [17]:
mytweets[1]._json['id'] #['text']

1378724057357578243

In [18]:
'in_reply_to_status_id' in mytweets[1]._json

True

In [81]:
def clean_tweet(x):
    try:
        clean = re.sub("@[A-Za-z0-9_]+","", x).strip()
        replytweet = re.match('… https://t.co/*', clean)
        if replytweet is not None:
            if replytweet.end() > 0:
                return None
        return clean
    except Exception as e:
        print(e)
        return x

In [206]:
%%time

dv = -1.0
dvd = {
    'toxicity': dv,
    'severe_toxicity': dv,
    'obscene': dv,
    'threat': dv,
    'insult': dv,
    'identity_hate': dv
}

res = []
tweeters = []
preds = []
for i in mytweets:
    if i._json['in_reply_to_status_id'] is not None:
        cleaned = clean_tweet(i._json['text'])
        res.append((i._json['id'], i._json['text'], cleaned))
        prediction = [dvd if cleaned is None else Detoxify('original').predict(cleaned)]
        tweeters.append(prediction)

CPU times: user 30min 4s, sys: 1min 3s, total: 31min 7s
Wall time: 56min 25s


In [221]:
len(predictions2['toxicity'])

746

In [208]:
res = []
for i in mytweets:
    if i._json['in_reply_to_status_id'] is not None:
        cleaned = clean_tweet(i._json['text'])
        res.append((i._json['id'], i._json['text'], cleaned))

In [211]:
cdf = pd.DataFrame(res, columns=['tweet_id', 'tweet', 'cleaned_tweet'])

In [216]:
%%time
predictions2 = Detoxify('original').predict(cdf[cdf['cleaned_tweet'].isnull()==False]['cleaned_tweet'].tolist())

CPU times: user 2min 59s, sys: 10.4 s, total: 3min 10s
Wall time: 53.1 s


In [165]:
tdfp.reset_index(drop=True).index

RangeIndex(start=0, stop=797, step=1)

In [ ]:
tdfp = pd.concat([pd.DataFrame(t) for t in tweeters], axis=0)

In [167]:
tdfp

In [168]:
tdf = pd.DataFrame(res, columns=['tweet_id', 'tweet'])
tdfp = pd.concat([pd.DataFrame(t) for t in tweeters], axis=0).reset_index(drop=True)
xvars = [
    'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult','identity_hate'
]
for x in xvars:
    tdf[x] = tdfp[x]

In [172]:
tdf.shape, tdfp.shape

((797, 8), (797, 6))

In [173]:
tdf.head()

,tweet_id,tweet,toxicity,severe_toxicity,obscene,threat,insult,identity_hate
0,1378516199642779650,@IsaiBCortez @cisco_iz @Eliana_Murillo @Buildw...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,1378511218994868228,@BuildwithJoy @Eliana_Murillo @EuniceCancino @...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2,1378509699104927744,@BuildwithJoy @Eliana_Murillo @EuniceCancino @...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
3,1378509579860930560,@tsawruk Chinese BBQ pork!!! Wife made this to...,0.252908,0.000787,0.012521,0.001147,0.013218,0.012737
4,1378508719349112832,@Eliana_Murillo @BuildwithJoy @EuniceCancino @...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


In [174]:
tdf['risk'] = tdf[xvars].max(axis=1)

In [178]:
tdf['risk'].describe()

count    797.000000
mean      -0.040707
std        0.271744
min       -1.000000
25%        0.000711
50%        0.001126
75%        0.003694
max        0.919482
Name: risk, dtype: float64

In [182]:
tdf.sort_values(by='risk', ascending=False)[['tweet_id', 'tweet', 'risk']]

,tweet_id,tweet,risk
270,1358441959384354816,"@IsaiBCortez Heck yeah, bro.\n\nThank God ever...",0.919482
658,1341018195709161473,@paul__132 Most code comments and documentatio...,0.892314
753,1334353023678574592,@MilesNextDoor bro u don't know how much i suc...,0.870974
737,1335728924018425856,"@chachovaladez Die Hard.\n\nJust kidding, it’s...",0.847634
745,1335019222057897984,"@mikulaja Also, that boss was obviously a jerk...",0.811857
...,...,...,...
172,1363085498164514818,@estebanuribe @EuniceCancino @Eliana_Murillo @...,-1.000000
173,1363085233944363008,@EuniceCancino @Eliana_Murillo @estebanuribe @...,-1.000000
174,1362967936508387332,@EuniceCancino @Eliana_Murillo @estebanuribe @...,-1.000000
175,1362965700701167617,@Eliana_Murillo @estebanuribe @EuniceCancino @...,-1.000000


# END